# Checking embeddings are collapsed & KNN

In [303]:
import numpy as np
import pandas as pd
import sys

embeddings = pd.read_pickle("embeddings.pkl")
ids = embeddings['Id'].values.astype('int')
embeddings = embeddings.drop(['Id'], axis=1).values
whales = np.load('raw_predictions.npy')

mean_emb = np.mean(np.concatenate((embeddings, whales), axis=0), axis=0)

In [282]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=16, metric='sqeuclidean', weights='distance')
#KNN = KNeighborsClassifier(n_neighbors=16, metric='sqeuclidean')
KNN.fit(embeddings, ids)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='sqeuclidean',
           metric_params=None, n_jobs=None, n_neighbors=16, p=2,
           weights='distance')

In [283]:
pred = KNN.predict_proba(embeddings)
predictions = np.argsort(-pred, axis=1)[:,:5] + 1

In [291]:
predictions[:5]

array([[   1, 3340, 3339, 3338, 3337],
       [   2,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338]], dtype=int64)

In [292]:
ids[:5]

array([1, 2, 3, 3, 3])

In [293]:
pred[:5]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [294]:
predictions[:5]

array([[   1, 3340, 3339, 3338, 3337],
       [   2,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338],
       [   3,    1, 3340, 3339, 3338]], dtype=int64)

In [295]:
KNN.predict(embeddings[:5])

array([1, 2, 3, 3, 3])

In [298]:
np.argmin(-pred[0]) + 1

1

In [290]:
np.where(pred[-1]>0)[0] + 1

array([5004], dtype=int64)

# New whale cutoff (~0.276)

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

embeddings = pd.read_pickle('embeddings.pkl')
whales = np.load('raw_predictions.npy', allow_pickle=True)

ids = embeddings['Id'].values.astype('int')
embeddings = embeddings.drop(['Id'], axis=1).values

KNN = KNeighborsClassifier(n_neighbors=5, metric='sqeuclidean', weights='distance', algorithm='brute')
KNN.fit(embeddings, ids)

dists, neighbours = KNN.kneighbors(whales, n_neighbors=200)
neighbours_labels = ids[neighbours.flat].reshape(neighbours.shape)

# get 5 nearest neighbours with different labels
predictions = np.zeros((len(whales), 5))
for i, labels in enumerate(neighbours_labels):
    j = 0
    prev_labels = []
    for label in labels:
        if label not in prev_labels:
            prev_labels.append(label)
            predictions[i, j] = label
            j += 1
        if j == 5:
            break

predictions = pd.DataFrame(data=predictions)

In [2]:
min_dists = dists[:, 0]

In [3]:
min(min_dists), max(min_dists)

(0.0005846443756000832, 0.28307008815890605)

In [22]:
#threshold = 0.0452
threshold = 0.015
new_whale_inds = np.where(min_dists >= threshold)[0]
len(new_whale_inds) / len(min_dists)

0.8346733668341708

In [23]:
len(new_whale_inds)

6644